In [1]:
# !pip install pandas sqlite3
# !pip install seaborn   
# !pip install plotly
# !pip install pandas
# !pip install matplotlib


In [2]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

sns.set(style="whitegrid")  # cleaner visuals


# Flight Delay Analysis: Identifying Root Causes from 3 Million US Flights

#### Project Goal: What causes the most flight delays?

This project analyzes over 3 million US domestic flights to uncover the top drivers of flight delays. Using SQL and Python, I examined trends across years, airports, airlines, and departure times. The analysis identifies peak delay periods, underperforming routes, and the impact of delay causes such as weather, air traffic, and airline issues. This actionable insight helps airlines and airports target improvements that can reduce disruptions.

In [3]:
df = pd.read_csv('data/flights_sample_3m.csv')
df


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,1522,JAX,"Jacksonville, FL",CLT,"Charlotte, NC",...,0.0,85.0,71.0,55.0,328.0,NaN,NaN,NaN,NaN,NaN
2999996,2022-11-02,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,1535,ORD,"Chicago, IL",AUS,"Austin, TX",...,0.0,176.0,145.0,130.0,977.0,NaN,NaN,NaN,NaN,NaN
2999997,2022-09-11,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2745,HSV,"Huntsville, AL",ATL,"Atlanta, GA",...,0.0,55.0,50.0,28.0,151.0,0.0,36.0,0.0,0.0,0.0
2999998,2019-11-13,Republic Airline,Republic Airline: YX,YX,20452,6134,BOS,"Boston, MA",LGA,"New York, NY",...,0.0,88.0,77.0,50.0,184.0,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()
df.isnull().sum()  # Count missing values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

FL_DATE                          0
AIRLINE                          0
AIRLINE_DOT                      0
AIRLINE_CODE                     0
DOT_CODE                         0
FL_NUMBER                        0
ORIGIN                           0
ORIGIN_CITY                      0
DEST                             0
DEST_CITY                        0
CRS_DEP_TIME                     0
DEP_TIME                     77615
DEP_DELAY                    77644
TAXI_OUT                     78806
WHEELS_OFF                   78806
WHEELS_ON                    79944
TAXI_IN                      79944
CRS_ARR_TIME                     0
ARR_TIME                     79942
ARR_DELAY                    86198
CANCELLED                        0
CANCELLATION_CODE          2920860
DIVERTED                         0
CRS_ELAPSED_TIME                14
ELAPSED_TIME                 86198
AIR_TIME                     86198
DISTANCE                         0
DELAY_DUE_CARRIER          2466137
DELAY_DUE_WEATHER   

In [5]:
conn = sqlite3.connect(':memory:')  # Creates a temporary SQLite database


In [6]:
df.to_sql('flight_data', conn, index=False, if_exists='replace')  # Save DataFrame as SQL table


3000000

## **SECTION-1: Delay Trends Over Time**

### 1. Total Number of Flights per Year

In [7]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year, COUNT(*) as Total_Flights
FROM flight_data
GROUP BY year;
'''
result1= pd.read_sql_query(query, conn)
result1


,Year,Total_Flights
0,2019,757673
1,2020,479350
2,2021,611633
3,2022,687860
4,2023,463484


- This query retrieves the total number of flights for each year by extracting the year from the flight date and grouping the data accordingly.
- It helps to understand the annual flight volume trends.

In [8]:
# Example: result from your 'total_flights.csv'
fig = px.bar(result1, x='Year', y='Total_Flights',
             title='Total Number of Flights per Year',
             labels={'Total_Flights': 'Flights'}
             )
fig.show()


- 2019 had the highest volume: ~758K flights.
- 2020 dropped sharply (~479K) due to COVID.
- Flight counts started recovering in 2021–2022, but 2023 dropped again, suggesting lingering effects or reduced capacity.

In [9]:
result1.to_csv('result1.csv', index=False)


### 2. On-Time vs Delayed Flights Yearly

In [10]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year,
       COUNT(CASE WHEN DEP_DELAY <= 0 THEN 1 END) as On_Time_Flights,
       COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) as Delayed_Flights,
       ROUND((COUNT(CASE WHEN DEP_DELAY <= 0 THEN 1 END) * 100.0 / COUNT(*)),3) as On_Time_Percentage,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Delayed_Percentage
FROM flight_data
GROUP BY year;
'''
result2= pd.read_sql_query(query, conn)
result2


,Year,On_Time_Flights,Delayed_Flights,On_Time_Percentage,Delayed_Percentage
0,2019,487494,257037,64.341,33.925
1,2020,364058,86649,75.948,18.076
2,2021,399254,202081,65.277,33.040
3,2022,407276,262563,59.209,38.171
4,2023,271431,184513,58.563,39.810


- This query identifies the number of on-time and delayed flights each year, along with their percentages.
- It highlights annual punctuality trends and the proportion of delayed flights.


In [11]:
result2.to_csv('result2.csv', index=False)


In [12]:
# Visualize yearly on-time vs delayed flights as a stacked bar chart
fig2 = px.bar(
    result2,
    x='Year',
    y=['On_Time_Flights', 'Delayed_Flights'],
    title='On-Time vs Delayed Flights per Year',
    labels={'value': 'Number of Flights', 'variable': 'Flight Status'},
    barmode='stack'
)
fig2.show()


### 3. Total Cancellations by Year

In [13]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year,
       COUNT(CASE WHEN CANCELLED=1 THEN 1 END) as Total_Cancellations,
       COUNT(*) as Total_Flights,
       ROUND((COUNT(CASE WHEN CANCELLED=1 THEN 1 END) * 100.0 / COUNT(*)),3) as Cancellation_Percentage
FROM flight_data
GROUP BY year;
'''
result3= pd.read_sql_query(query, conn)
result3


,Year,Total_Cancellations,Total_Flights,Cancellation_Percentage
0,2019,13594,757673,1.794
1,2020,28757,479350,5.999
2,2021,10532,611633,1.722
3,2022,18448,687860,2.682
4,2023,7809,463484,1.685


- This query calculates the total number of cancellations, total flights, and the percentage of cancellations for each year.
- It gives insights into how cancellations have varied over time and their proportion relative to total flights.

In [14]:
result3.to_csv('result3.csv', index=False)



In [15]:
# Visualize yearly cancellations as a bar chart
fig3 = px.bar(
    result3,
    x='Year',
    y='Total_Cancellations',
    title='Total Flight Cancellations per Year',
    labels={'Total_Cancellations': 'Number of Cancellations'}
)
fig3.show()


* **2020** saw a **massive spike** in cancellations: **28,757 flights**, nearly **6% of all scheduled flights**.
* Post-COVID, cancellation rates stabilized under 2% by 2023.

### 4. Average Delay Time by Year

In [16]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year,
       AVG(DEP_DELAY) as Average_Departure_Delay,
       AVG(ARR_DELAY) as Average_Arrival_Delay
FROM flight_data
WHERE DEP_DELAY > 0 OR ARR_DELAY > 0
GROUP BY year;
'''
result4= pd.read_sql_query(query, conn)
result4


,Year,Average_Departure_Delay,Average_Arrival_Delay
0,2019,30.944973,30.114348
1,2020,21.970824,21.900776
2,2021,28.634240,26.795731
3,2022,31.294262,29.589928
4,2023,34.285028,33.370908


- This query calculates the average delay times for departures and arrivals annually.
- It excludes flights without delays, providing a focused view of delay durations.

In [17]:
result4.to_csv('result4.csv', index=False)



In [18]:
# Visualize average departure and arrival delay by year
fig4 = px.bar(
    result4,
    x='Year',
    y=['Average_Departure_Delay', 'Average_Arrival_Delay'],
    title='Average Departure and Arrival Delay by Year',
    labels={'value': 'Average Delay (minutes)', 'variable': 'Delay Type'},
    barmode='group'
)
fig4.show()


* Both **arrival and departure delays peaked in 2023**:

  * Avg Departure: **\~34.3 min**
  * Avg Arrival: **\~33.4 min**
* Delay duration increased steadily post-2020.

**Insight**: Despite lower flight volumes in 2023, **delay rates and durations increased**, suggesting operational or systemic inefficiencies rather than demand-driven issues.

# **SECTION-2: Delay Causes**

In [19]:
query = '''
SELECT AIRLINE, ORIGIN, DEST,
       COUNT(CASE WHEN DELAY_DUE_CARRIER > 0 THEN 1 END) AS Carrier_Delays,
       COUNT(CASE WHEN DELAY_DUE_WEATHER > 0 THEN 1 END) AS Weather_Delays,
       COUNT(CASE WHEN DELAY_DUE_NAS > 0 THEN 1 END) AS NAS_Delays,
       COUNT(CASE WHEN DELAY_DUE_SECURITY > 0 THEN 1 END) AS Security_Delays,
       COUNT(CASE WHEN DELAY_DUE_LATE_AIRCRAFT > 0 THEN 1 END) AS Late_Aircraft_Delays,
       COUNT(*) AS Total_Flights
FROM flight_data
GROUP BY AIRLINE, ORIGIN, DEST
ORDER BY Total_Flights DESC;

'''
result5= pd.read_sql_query(query, conn)
result5


,AIRLINE,ORIGIN,DEST,Carrier_Delays,Weather_Delays,NAS_Delays,Security_Delays,Late_Aircraft_Delays,Total_Flights
0,Hawaiian Airlines Inc.,OGG,HNL,517,41,105,2,522,3577
1,Hawaiian Airlines Inc.,HNL,OGG,368,27,31,1,221,3496
2,Alaska Airlines Inc.,SEA,ANC,277,15,353,2,173,2474
3,Hawaiian Airlines Inc.,HNL,KOA,242,19,10,2,157,2467
4,Hawaiian Airlines Inc.,LIH,HNL,239,28,75,0,294,2459
...,...,...,...,...,...,...,...,...,...
17814,United Air Lines Inc.,SJC,LAS,0,0,0,0,1,1
17815,United Air Lines Inc.,SMF,LAX,0,0,1,0,0,1
17816,United Air Lines Inc.,SRQ,IAH,0,0,0,0,0,1
17817,United Air Lines Inc.,TVC,EWR,0,0,0,0,0,1


- This query analyzes delay reasons by categorizing them into carrier, weather, NAS (National Airspace System), security, and late aircraft delays.
- It helps identify patterns and key contributors to delays.


In [20]:
result5.to_csv('result5.csv', index=False)



In [ ]:
# Load your data
df = pd.read_csv("result5.csv")

# === DESTINATION AIRPORTS ===
dest_group = df.groupby('DEST')[['Carrier_Delays', 'Weather_Delays', 'NAS_Delays', 'Security_Delays', 'Late_Aircraft_Delays']].sum()
dest_group['Total'] = dest_group.sum(axis=1)
dest_top10 = dest_group.sort_values(by='Total', ascending=False).head(10).reset_index()
dest_long = dest_top10.melt(id_vars='DEST', var_name='Delay Cause', value_name='Count')

fig_dest = px.bar(dest_long, x='DEST', y='Count', color='Delay Cause',
                  title='Top 10 Routes: Delay Causes by Destination Airport',
                  labels={'DEST': 'Destination Airport'},
                  barmode='group')
fig_dest.show()

# === ORIGIN AIRPORTS ===
origin_group = df.groupby('ORIGIN')[['Carrier_Delays', 'Weather_Delays', 'NAS_Delays', 'Security_Delays', 'Late_Aircraft_Delays']].sum()
origin_group['Total'] = origin_group.sum(axis=1)
origin_top10 = origin_group.sort_values(by='Total', ascending=False).head(10).reset_index()
origin_long = origin_top10.melt(id_vars='ORIGIN', var_name='Delay Cause', value_name='Count')

fig_origin = px.bar(origin_long, x='ORIGIN', y='Count', color='Delay Cause',
                    title='Top 10 Routes: Delay Causes by Origin Airport',
                    labels={'ORIGIN': 'Origin Airport'},
                    barmode='group')
fig_origin.show()

# === AIRLINES ===
airline_group = df.groupby('AIRLINE')[['Carrier_Delays', 'Weather_Delays', 'NAS_Delays', 'Security_Delays', 'Late_Aircraft_Delays']].sum()
airline_group['Total'] = airline_group.sum(axis=1)
airline_top10 = airline_group.sort_values(by='Total', ascending=False).head(10).reset_index()
airline_long = airline_top10.melt(id_vars='AIRLINE', var_name='Delay Cause', value_name='Count')

fig_airline = px.bar(airline_long, x='AIRLINE', y='Count', color='Delay Cause',
                     title='Top 10 Routes: Delay Causes by Airline',
                     labels={'AIRLINE': 'Airline'},
                     barmode='group')
fig_airline.show()


**Delay Causes Breakdown**

 While “Carrier” and “Late Aircraft” were dominant causes overall, a deeper breakdown reveals location-specific challenges:

 * **DFW** suffers across all categories, especially carrier and airspace (NAS) delays.
 * **HNL, JFK, and LAX** show intense carrier-related delay clusters.
 * **American Airlines** is the largest contributor to delays across **carrier, NAS, and aircraft** issues, while **Alaska and Hawaiian** face fewer issues overall.

 These insights can guide route-specific improvement strategies and airline accountability.



**Insight**: Most delays are **controllable**, coming from the airlines or previous flights — not external systems.

# **SECTION-3: Airport & Airline Bottlenecks**

### 1. Top 10 airports with highest departure delay %

In [22]:
query = '''
SELECT ORIGIN,
       ORIGIN_CITY,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Departure_Delay_Percentage
FROM flight_data
GROUP BY ORIGIN
ORDER BY Departure_Delay_Percentage DESC
LIMIT 10;
'''
result6= pd.read_sql_query(query, conn)
result6


,ORIGIN,ORIGIN_CITY,Departure_Delay_Percentage
0,CDB,"Cold Bay, AK",71.429
1,OGD,"Ogden, UT",55.556
2,HGR,"Hagerstown, MD",54.945
3,PPG,"Pago Pago, TT",54.839
4,MDW,"Chicago, IL",51.674
5,ADK,"Adak Island, AK",51.220
6,HTS,"Ashland, WV",50.855
7,DAL,"Dallas, TX",50.485
8,SCK,"Stockton, CA",50.000
9,HOU,"Houston, TX",48.784


- This query identifies the top 10 airports with the highest departure delay percentages.
- It focuses on airports contributing significantly to delays.

In [23]:
result6.to_csv('result6.csv', index=False)


In [24]:
# Visualize the top 10 airports with the highest departure delay percentage
fig_airport_delay = px.bar(
    result6,
    x='ORIGIN',
    y='Departure_Delay_Percentage',
    title='Top 10 Airports with Highest Departure Delay Percentage',
    labels={'Departure_Delay_Percentage': 'Departure Delay (%)', 'ORIGIN': 'Airport Code'}
)
fig_airport_delay.show()


* **Cold Bay, AK (CDB)** tops the list with **71.4% delayed departures**
* Major metros like **Chicago (MDW)** and **Houston (HOU)** also appear with **\~50%+** delay rates.

### 2. Airline Delay Ranking

In [25]:
query = '''
SELECT DOT_CODE AS UNIQUE_CARRIER,
        AIRLINE,
        AVG(DEP_DELAY) as Average_Departure_Delay,
        AVG(ARR_DELAY) as Average_Arrival_Delay
FROM flight_data
GROUP BY UNIQUE_CARRIER
ORDER BY Average_Departure_Delay DESC;
'''
result7= pd.read_sql_query(query, conn)
result7


,UNIQUE_CARRIER,AIRLINE,Average_Departure_Delay,Average_Arrival_Delay
0,20409,JetBlue Airways,18.322555,12.276125
1,20436,Frontier Airlines Inc.,16.033574,11.100429
2,20368,Allegiant Air,13.907797,13.284601
3,20416,Spirit Air Lines,12.981880,8.029485
4,20366,ExpressJet Airlines LLC d/b/a aha!,12.774462,10.031976
5,19805,American Airlines Inc.,12.609895,6.661229
6,20378,Mesa Airlines Inc.,12.277383,7.309810
7,19977,United Air Lines Inc.,11.217032,5.035985
8,19393,Southwest Airlines Co.,10.816957,3.269796
9,20304,SkyWest Airlines Inc.,9.458294,4.023311


- This query ranks airlines based on their average departure and arrival delays, providing a performance comparison among carriers.


In [26]:
result7.to_csv('result7.csv', index=False)



In [ ]:
# Visualize average departure delay by airline
fig = px.bar(
    result7,
    y='AIRLINE',
    x='Average_Departure_Delay',
    orientation='h',
    title='Average Departure Delay by Airline',
    labels={'Average_Departure_Delay': 'Average Departure Delay (minutes)', 'AIRLINE': 'Airline'},
    color='Average_Departure_Delay'
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()


* **JetBlue** has the highest average departure delay: **\~18.3 mins**
* **Frontier, Allegiant, Spirit** also have high delay times.
* Top performers:

  * **Alaska Airlines**: \~4.6 mins
  * **Delta** and **Hawaiian**: < 8 mins

### 3. Cancellations by Airline

In [28]:
query = '''
SELECT DOT_CODE AS UNIQUE_CARRIER,
        AIRLINE,
       ROUND((COUNT(CASE WHEN CANCELLED=1 THEN 1 END) * 100.0 / COUNT(*)),3) as Cancellation_Percentage
FROM flight_data
GROUP BY UNIQUE_CARRIER
ORDER BY Cancellation_Percentage DESC;
'''
result8= pd.read_sql_query(query, conn)
result8


,UNIQUE_CARRIER,AIRLINE,Cancellation_Percentage
0,20366,ExpressJet Airlines LLC d/b/a aha!,5.565
1,20368,Allegiant Air,4.519
2,20378,Mesa Airlines Inc.,3.650
3,19393,Southwest Airlines Co.,3.377
4,20452,Republic Airline,3.247
5,20397,PSA Airlines Inc.,3.084
6,20398,Envoy Air,2.996
7,19805,American Airlines Inc.,2.847
8,20409,JetBlue Airways,2.693
9,20436,Frontier Airlines Inc.,2.584


- This query calculates the cancellation percentage for each airline, ranking them to highlight the carriers with the highest and lowest cancellation rates.

In [29]:
result8.to_csv('result8.csv', index=False)


In [30]:
# Visualize airline cancellation percentage as a horizontal bar chart
fig = px.bar(
    result8,
    y='AIRLINE',
    x='Cancellation_Percentage',
    orientation='h',
    title='Flight Cancellation Percentage by Airline',
    labels={'Cancellation_Percentage': 'Cancellation Percentage (%)', 'AIRLINE': 'Airline'}
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()


* Worst:

  * **ExpressJet**: **5.56%** cancellations
  * **Allegiant**, **Mesa**, **Southwest** all above 3.3%
* Best:

  * **Hawaiian Airlines**: **1.2%**
  * **Delta**: **1.5%**


**Insight**: Ultra-low-cost and regional carriers tend to perform worse on both delays and cancellations. Legacy carriers show stronger reliability.

# **SECTION-4: High-Risk Routes**

### 1.Top Delayed Flight Routes

In [31]:
query = '''
SELECT ORIGIN || '-' || DEST as Route,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Departure_Delay_Percentage,
       ROUND((COUNT(CASE WHEN ARR_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Arrival_Delay_Percentage
FROM flight_data
GROUP BY Route
ORDER BY Departure_Delay_Percentage DESC
LIMIT 10;
'''
result9= pd.read_sql_query(query, conn)
result9


,Route,Departure_Delay_Percentage,Arrival_Delay_Percentage
0,VPS-TOL,100.0,100.000
1,TWF-SUN,100.0,0.000
2,TWF-SFO,100.0,100.000
3,TPA-ORF,100.0,0.000
4,TOL-VPS,100.0,50.000
5,SMF-BNA,100.0,83.333
6,SJU-CVG,100.0,100.000
7,SJC-RDU,100.0,0.000
8,SEA-HOU,100.0,71.429
9,SDF-SFO,100.0,100.000


- This query identifies the top 10 flight routes with the highest percentages of departure and arrival delays, highlighting problematic routes.


In [32]:
result9.to_csv('result9.csv', index=False)


In [33]:
# Visualize the top 10 delayed flight routes by departure delay percentage
fig_routes_dep = px.bar(
    result9,
    x='Route',
    y='Departure_Delay_Percentage',
    title='Top 10 Flight Routes with Highest Departure Delay Percentage',
    labels={'Departure_Delay_Percentage': 'Departure Delay (%)', 'Route': 'Flight Route'}
)
fig_routes_dep.show()

# Visualize the top 10 delayed flight routes by arrival delay percentage
fig_routes_arr = px.bar(
    result9,
    x='Route',
    y='Arrival_Delay_Percentage',
    title='Top 10 Flight Routes with Highest Arrival Delay Percentage',
    labels={'Arrival_Delay_Percentage': 'Arrival Delay (%)', 'Route': 'Flight Route'}
)
fig_routes_arr.show()


* Every route listed has **100% departure delay percentage**!
* Routes like **SEA–HOU**, **SJU–CVG**, and **TOL–VPS** show consistent disruptions.
* Some routes show **100% departure delay but 0% arrival delay**, suggesting time padding or recovery mid-air.

### 2. Impact of Scheduled Flight Time on Delays

In [34]:
query = '''
SELECT CRS_DEP_TIME,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Departure_Delay_Percentage,
       ROUND((COUNT(CASE WHEN ARR_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Arrival_Delay_Percentage
FROM flight_data
GROUP BY CRS_DEP_TIME
ORDER BY CRS_DEP_TIME;
'''
result10= pd.read_sql_query(query, conn)
result10


,CRS_DEP_TIME,Departure_Delay_Percentage,Arrival_Delay_Percentage
0,1,54.167,54.167
1,2,40.000,20.000
2,3,100.000,100.000
3,4,58.824,41.176
4,5,38.971,25.000
...,...,...,...
1379,2355,36.000,31.875
1380,2356,40.580,27.053
1381,2357,49.198,41.711
1382,2358,47.143,36.190


- This query examines how scheduled departure times (CRS_DEP_TIME) influence the percentage of delays for departures and arrivals.
- It provides insights into delay patterns across different times of the day.


In [35]:
result10.to_csv('result10.csv', index=False)


In [36]:
fig_time_delay = px.line(
    result10,
    x = 'CRS_DEP_TIME',
    y = ['Departure_Delay_Percentage', 'Arrival_Delay_Percentage'],
    title = 'Impact of Scheduled Departure Time on Delay Percentages',
    labels = {
        'CRS_DEP_TIME': 'Scheduled Departure Time',
        'value': 'Delay Percentage (%)',
        'variable': 'Delay Type'
    }
)
fig_time_delay.show()


* Across **1384 unique scheduled departure times**, delays show a spike during certain hours.
* Time slots like **3 AM, 4 AM, 11 PM–midnight** show high delay percentages (>50%).
* However, most extreme delay windows are **small-volume slots**, suggesting noise in some time ranges.

**Insight**: Departure delays are **not uniformly distributed across the day** — targeted slot analysis could help optimize scheduling and staffing.